# SSB Kayıt 


## Step 1: Waiting for the Next Period Reading
- **State Number**: 1
- **State Code**: -
- **State Name**: Waiting for the Next Period Reading
- **Description**: This step waits for the periodic reading after the SSB registration date to trigger the creation of accrual. The system listens for an event (`thor-billing-accrual-created`) to confirm the accrual data and proceeds based on this information.
- **Status**: Pending

#### 1A - Çalışma öncesi Script

In [ ]:
# machine                     : Currently executing machine instance
# state                       : Currently executing state instance
# request_data                : Original trigger data
# context                     : Machine stored context (1. Muhatap(ID), 2. Tesisat(ID), 3. Sözleşme)
# state_responses_of_machine  : State responses of Machine <StateCode, {prev, current, webhook}>
# return
#     json input of business logic service
import json
from datetime import datetime
def state_handler(machine, state, request_data, context, state_response_of_machine):
    cancelDate = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode'] == 'ContractCancellationDate'), None)['value']
    if cancelDate != 99991231595959:
        return {"success": True, "error": {"code": "END", "detail": "İptal Tarihi"}}


#### 1B -  İstek Eşleştirme Scripti

In [ ]:
#Null

#### 1C - State Context Değer Tutma 

In [9]:
# machine                     : Currently executing machine instance
# state                       : Currently executing state instance
# request_data                : Original trigger data
# context                     : Machine stored context (1. Muhatap(ID), 2. Tesisat(ID), 3. Sözleşme)
# state_responses_of_machine  : State responses of Machine <StateCode, {prev, current, webhook}>
# return
#     json input of business logic service
import json
from datetime import datetime
def state_handler(machine, state, request_data, context, state_response_of_machine):
    current_state_response = state_response_of_machine
    store = {'AccrualDate':current_state_response['AccrualDate'],
             'Endeks3':current_state_response['Endeks3'],
             'Endeks2':current_state_response['Endeks2'],
             'Endeks1':current_state_response['Endeks1']
             }
    
    return store

#### 1D - Çalışma Sonrası Script

#### 1E - İş Mantığı

In [7]:
"""event/thor-billing-accrual-create"""

'event/thor-billing-accrual-create'

## Step 2: Contract Terminated
- **State Number**: 2
- **State Code**: -
- **State Name**: Contract Terminated
- **Description**: This step ensures that the contract is terminated under specific rules for different subscriber groups. If the contract is not renewed, it terminates based on the subscriber group being "Industrial." Public and private service sectors contracts are also considered for termination.
- **Status**: Done

#### 2A - Çalışma Öncesi Scripti

In [ ]:
# machine                     : Currently executing machine instance
# state                       : Currently executing state instance
# request_data                : Original trigger data
# context                     : Machine stored context (1. Muhatap(ID), 2. Tesisat(ID), 3. Sözleşme)
# state_responses_of_machine  : State responses of Machine <StateCode, {prev, current, webhook}>
# return
#     json input of business logic service
import json
from datetime import datetime
def state_handler(machine, state, request_data, context, state_response_of_machine):
    
    current_date = datetime.now().strftime('%Y%m%d') + '000000'

    
    subscriberGroup = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode']=='SubscriberGroup'),None)['values'][0]['value']
    ssbDocEndDate = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode']=='SsbDocumentEndDate'),None)['value']
    contractClass = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode']=='ContractClass'),None)['value']

    if contractClass  not in ['K2ST', 'DST']:
        return {"success": True, "error": {"code": "END", "detail": "ContractClass"}}

    
    if request_data['isNew'] == True and subscriberGroup == 'SA':
        if ssbDocEndDate > current_date:
            return {"success": True, "goto": "x"}
        else:
            return {"success": True, "error": {"code": "END", "detail": "SsbDocumentEndDate"}}

    elif request_data['isNew'] == False and subscriberGroup == 'KO':
        return {"success": True, "goto": "x"}
    else :
        return {"success": True, "error": {"code": "END", "detail": "Subscriber Group"}}



#### 2B - İstek Eşleştirme Scripti

In [5]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
from datetime import datetime

def state_handler(machine, state, request_data, context, state_response_of_machine) -> json:  
    
    current_date = datetime.now().strftime('%Y%m%d') + '000000'
    
    data = {
        "id": request_data["id"],
        "attributes": [
            {
                "attribute": {
                    "attributeCode": "ContractCancellationDate"
                },
                "value":current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationRequestDate"
                },
                "value": current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationReason"
                },
                "value": "SubscriberRequest"  
            },
            {
                "attribute": {
                    "attributeCode": "ContractNumber"
                },
                "value": next((attr['value'] for attr in request_data['attributes'] 
                               if attr['attribute']['attributeCode'] == "ContractNumber"), None)
            }
        ],
        "authorityGroup": request_data["authorityGroup"],
        "contractLockID": "string"  
    }
    
    return json.dumps(data, indent=4)


#### 2C - State Context Değer Tutma Scripti

In [ ]:
#Null

#### 2D - Çalışma Sonrası Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping

import json

def state_handler(machine, state, request_data, context, state_response_of_machine):

    return {"success": True, "goto": "X"}

#### 2E - İş Mantığı

In [10]:
"""contract/cancel"""

'contract/cancel'

## Step 3: Data Preparation for New Account
- **State Number**: 3
- **State Code**: -
- **State Name**: Data Preparation for New Account
- **Description**: In this step, the system prepares the data needed to create a new account. The necessary attribute values are gathered from the terminated contract and associated account details.
- **Status**: Pending

#### 3A - Çalışma Öncesi Script

In [11]:
#Null

#### 3B - İstek Eşleştirme Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
from datetime import datetime

def state_handler(machine, state, request_data, context, state_response_of_machine) -> json:  
    contractAccountId = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode']=='ContractAccountId'),None)['value']

    return {"assetid":contractAccountId}

#### 3C - State Context Değer Tutma Scripti

In [ ]:
#Null

#### 3D - Çalışma Sonrası Scripti 

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping

import json

def state_handler(machine, state, request_data, context, state_response_of_machine):

    return {"success": True, "goto": "X"}

#### 3E - İş Mantığı

In [ ]:
"""md/search"""

## Step 4: New Account Created
- **State Number**: 4
- **State Code**: -
- **State Name**: New Account Created
- **Description**: This step creates a new account based on the details of the terminated contract. Some attribute values may differ, and these changes are reflected in the new account. Historical data for the old account is also updated to match the new expiry dates.
- **Status**: Pending


#### 4A. Çalışma Öncesi Scripti

In [ ]:
#Null

#### 4B. İstek Eşleştirme Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
from datetime import datetime

def state_handler(machine, state, request_data, context, state_response_of_machine):

    prev_state_response = state_response_of_machine
    
    create_date = next((attr for attr in prev_state_response['attributes'] if attr['attribute']['attributeCode'] == 'CreateDate'),None)['value'] 
    
    AccountClassCode = 1
    TaxClass = 2
    AccountGLCode = 3
    AccountSubscriberGroup = 4
    
    changes_dict = {'AccountClassCode': AccountClassCode,
                    'TaxClass': TaxClass,
                    'AccountGLCode': AccountGLCode,
                    'ExpiryDate':create_date
                    }
    
    
    if AccountSubscriberGroup == 'IndustrySubscriberGroup':
        changes_dict['AccountSubscriberGroup'] = 'PAPSSAOSG'
    else:
        changes_dict['AccountSubscriberGroup'] = 'IndustrySubscriberGroup'
    
    copied_json = {}
    stack = [(copied_json, prev_state_response)]

    # Copy context to copied_json without id
    while stack:
        current_copied, current_original = stack.pop()

        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value
    
    for code, new_value in changes_dict.items():
        for attr in copied_json.get('attributes', []):
            if attr['attribute']['attributeCode'] == code:
                if attr['value'] is None:
                    attr['values'][0]['value'] = new_value
                    attr['values'][0]['valueDesc'] = None
                else:
                    attr['values'][0]['valueDesc'] = None
                    attr['value'] = new_value
    
    return copied_json


#### 4C. State Context Değer Tutma Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping

import json

def state_handler(machine, state, request_data, context, state_response_of_machine):
    current_state_response = state_response_of_machine
    AccountId = state_response_of_machine['id']

    store = {'NewAccountId':AccountId}
    return store

#### 4D. Çalışma Sonrası Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping

import json

def state_handler(machine, state, request_data, context, state_response_of_machine):

    return {"success": True, "goto": "X"}

#### 4E. İş Mantığı

In [ ]:
"""account/save"""

'md/save'

## Step 5: New Contract Created
- **State Number**: 5
- **State Code**: -
- **State Name**: New Contract Created
- **Description**: A new contract is created using the data from the terminated contract. If the contract was not renewed, this step ensures the creation of a contract for the "Industrial" subscriber group. For public and private service sectors, the contract is generated with new attributes like ContractAccountId, SubscriberGroup, and ContractBeginDate.
- **Status**: Pending

#### 5A. Çalışma Öncesi Scripti

In [ ]:
#Null

#### 5B. İstek Eşleştirme Scripti

In [ ]:
from datetime import datetime
import json
def state_handler(machine, state, request_data, context, state_response_of_machine):
    # Not recursive 
    prev_state_response = state_response_of_machine
    
    current_date = datetime.now().strftime('%Y%m%d') + '000000'
    #create_date = next((attr for attr in prev_state_response['attributes'] if attr['attribute']['attributeCode'] == 'CreateDate'),None)['value']        
    
    prev_state_response = state_response_of_machine
    subscriberGroup = 2
    contractBeginReason = 1
    SsbDocumentId = 3
    
    changes_dict = {'TermType': 'ÇT',
                    'ProducerTermType': 'ÇT',
                    'ContractAccountId': prev_state_response['id'],
                    'ProducerAccountId' : prev_state_response['id'],
                    'ContractBeginReason':contractBeginReason,
                    'SsbDocumentId' : SsbDocumentId,
                    'ContractSignDate': current_date,
                    'ContractBeginDate' : current_date
                    }
    
    if subscriberGroup == 'KO':
        changes_dict['SubscriberGroup'] = 'SA'
    else:
        changes_dict['SubscriberGroup'] = 'KO'
    
    copied_json = {}
    stack = [(copied_json, request_data)]

    # Copy context to copied_json without id
    while stack:
        current_copied, current_original = stack.pop()

        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value

    # Set dates for all values in copied_json
    for key, value in copied_json.items():
        if isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    if item.get('value') is None and 'values' in item and item['values']:
                        item['values'][0]['startDate'] = current_date
                        item['values'][0]['endDate'] = 99991231235959
                    elif item.get('value') is not None:
                        item['startDate'] = current_date
                        item['endDate'] = 99991231235959
        elif isinstance(value, dict):
            if value.get('value') is None and 'values' in value and value['values']:
                value['values'][0]['startDate'] = current_date
                value['values'][0]['endDate'] = 99991231235959
            elif value.get('value') is not None:
                value['startDate'] = current_date
                value['endDate'] = 99991231235959

    # Update attribute values in context according to the dict above
    for code, new_value in changes_dict.items():
        for attr in copied_json.get('attributes', []):
            if attr['attribute']['attributeCode'] == code:
                if attr['value'] is None:
                    attr['values'][0]['value'] = new_value
                    attr['values'][0]['valueDesc'] = None
                else:
                    attr['values'][0]['valueDesc'] = None
                    attr['value'] = new_value

    return copied_json  

#### 5C. State Context Değer Tutma Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping

import json

def state_handler(machine, state, request_data, context, state_response_of_machine):
    current_state_response = state_response_of_machine
    ContractId = state_response_of_machine['id']

    store = {'NewContractId':ContractId}
    return store

#### 5D. Çalışma Sonrası Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping

import json

def state_handler(machine, state, request_data, context, state_response_of_machine):

    return {"success": True, "goto": "X"}

#### 5E. İş Mantığı

In [ ]:
"""contract/create"""

## Step 6: Closing Invoice Created
- **State Number**: 6
- **State Code**: -
- **State Name**: Closing Invoice Created
- **Description**: This step ensures that the closing invoice is generated using the appropriate rate for the subscriber group (either public/private service sectors or industrial). The closing invoice is calculated using the accrual date and reading indexes.
- **Status**: Pending

#### 6A - Çalışma Öncesi Scripti

In [ ]:
#Null

#### 6B - İstek Eşleştirme Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping

import json

def state_handler(machine, state, request_data, context, state_response_of_machine):
    return{context[['AccrualDate','Endeks1','Endeks2','Endeks3']]}

#### 6C - State Context Değer Tutma Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping

import json

def state_handler(machine, state, request_data, context, state_response_of_machine):
    store = {}
    return store

#### 6D - Çalışma Sonrası Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping

import json

def state_handler(machine, state, request_data, context, state_response_of_machine):

    return {"success": True, "goto": "END"}

#### 6E - İş Mantığı

In [ ]:
"""accruals/calculate"""